In [1]:
import openai

In [2]:
from openai import OpenAI

In [3]:
client = OpenAI()

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [5]:
response = client.chat.completions.create(
    model= "gpt-3.5-turbo-16k",
    messages=[{"role": "user",
               "content": "is it too late to join the course?"}])

In [14]:
response.choices[0].message.content

"I'm sorry, but I need more information about which course you are referring to in order to provide an accurate answer. Can you please provide more context?"

In [24]:
import os

os.environ['OPENAI_API_KEY']

environ{'SHELL': '/bin/bash',
        'NUGET_XMLDOC_MODE': 'skip',
        'COLORTERM': 'truecolor',
        'CLOUDENV_ENVIRONMENT_ID': '0fcc7025-eab9-4503-b524-755945cd425c',
        'NVM_INC': '/usr/local/share/nvm/versions/node/v20.14.0/include/node',
        'TERM_PROGRAM_VERSION': '1.88.1',
        'GITHUB_USER': 'daandeey',
        'rvm_prefix': '/usr/local',
        'CODESPACE_NAME': 'ubiquitous-acorn-vgrp6q45vq436g77',
        'HOSTNAME': 'codespaces-22ac95',
        'JAVA_ROOT': '/home/codespace/java',
        'JAVA_HOME': '/usr/local/sdkman/candidates/java/current',
        'DOTNET_ROOT': '/usr/share/dotnet',
        'CODESPACES': 'true',
        'PYTHON_ROOT': '/home/codespace/.python',
        'GRADLE_HOME': '/usr/local/sdkman/candidates/gradle/current',
        'NVS_DIR': '/usr/local/nvs',
        'NVS_OS': 'linux',
        'DOTNET_SKIP_FIRST_TIME_EXPERIENCE': '1',
        'MY_RUBY_HOME': '/usr/local/rvm/rubies/ruby-3.2.4',
        'NVS_USE_XZ': '1',
        'SDKMAN_CANDID